In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import sklearn
from datetime import date
from datetime import datetime
import math

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

In [3]:
tickets_bare = pd.read_csv("../../Full_Data/tickets_bare_full.csv")


In [4]:
tickets_bare['event_name'].unique()

array(['CLT21HOU', 'CLT21JAX', 'CLT21LAR', 'CLT21NE', 'CLT21NYJ',
       'CLT21PRE', 'CLT21SEA', 'CLT21TB', 'CLT21TEN', 'CLT22JAX',
       'CLT22KC', 'CLT22LAC', 'CLT22PHI', 'CLT22PIT', 'CLT22PR1',
       'CLT22PR2', 'CLT22TEN', 'CLT22WAS', 'CLT21LV', 'CLT22HOU'],
      dtype=object)

In [ ]:
tickets_cleaned = pd.read_csv("../../Full_Data/tickets_cleaned_full.csv")


In [5]:
tickets_bare[['isAttended']] = tickets_bare[['isAttended']].replace(1,0)
tickets_bare[['isAttended']] = tickets_bare[['isAttended']].replace(2,1)

In [6]:
tickets_cleaned[['isAttended']] = tickets_cleaned[['isAttended']].replace(1,0)
tickets_cleaned[['isAttended']] = tickets_cleaned[['isAttended']].replace(2,1)

In [7]:
tickets_cleaned.event_name.unique()

array(['CLT21HOU', 'CLT21JAX', 'CLT21LAR', 'CLT21NE', 'CLT21NYJ',
       'CLT21PRE', 'CLT21SEA', 'CLT21TB', 'CLT21TEN', 'CLT22JAX',
       'CLT22KC', 'CLT22LAC', 'CLT22PHI', 'CLT22PIT', 'CLT22PR1',
       'CLT22PR2', 'CLT22TEN', 'CLT22WAS', 'CLT21LV', 'CLT22HOU'],
      dtype=object)

In [8]:
tickets_bare.head()
# 1 means yes, 0 means no

,Unnamed: 0,event_name,Price,Term,SeatUniqueID,Tenure,UniqueID,isAttended,event_date_0,event_date_1,...,Saturday.1,Sunday,Sunday,Thursday,Sun_No,Sun_Yes,Mercch_Yes,Merch_No,other_events_No,other_events_Yes
0,0,CLT21HOU,-0.098213,0.0,101-10-1,25.0,CLT21HOU-101-10-1,0,0.002213,0.000006,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
1,1,CLT21HOU,-0.098213,0.0,101-10-10,20.0,CLT21HOU-101-10-10,0,0.002213,0.000006,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,2,CLT21HOU,-0.098213,0.0,101-10-11,20.0,CLT21HOU-101-10-11,0,0.002213,0.000006,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
3,3,CLT21HOU,-0.098213,0.0,101-10-12,4.0,CLT21HOU-101-10-12,0,0.002213,0.000006,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
4,4,CLT21HOU,-0.098213,0.0,101-10-13,32.0,CLT21HOU-101-10-13,0,0.002213,0.000006,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0


In [9]:
tickets_cleaned.head()

,Unnamed: 0,Club,Comp,Employee,Group,Group NEW,Hold,Hold Account,House,Huntington,...,Saturday.1,Sunday,Sunday,Thursday,Sun_No,Sun_Yes,Mercch_Yes,Merch_No,other_events_No,other_events_Yes
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0


In [10]:
tickets_cleaned_extra_copy = tickets_cleaned.copy()

In [11]:
tickets_cleaned = tickets_cleaned.sample(frac= 0.00005, replace= False, random_state= 1234)

### Splitting the data

In [12]:
test = tickets_cleaned[(tickets_cleaned['event_name'] == 'CLT21LV') | tickets_cleaned['event_name'] == 'CLT22HOU']
train = tickets_cleaned[(tickets_cleaned['event_name'] != 'CLT21LV') & tickets_cleaned['event_name'] != 'CLT22HOU']

In [9]:
#test = tickets_cleaned[(tickets_cleaned['event_name'] == 'CLT21JAX')]
#train = tickets_cleaned[(tickets_cleaned['event_name'] != 'CLT21LV') & (tickets_cleaned['event_name'] != 'CLT22HOU') & (tickets_cleaned['event_name'] != 'CLT21JAX')]

In [13]:
test_UniqueID = test[['UniqueID', 'isAttended']]

- Must drop the uncodable data

In [14]:
train = train.drop(labels = ['event_name', 'UniqueID', 'Unnamed: 0', 'SeatUniqueID'], axis= 1)
test = test.drop(labels = ['event_name', 'UniqueID', 'Unnamed: 0', 'SeatUniqueID'], axis = 1)

In [15]:
y_train = train[['isAttended']]
y_test = test[['isAttended']]

In [16]:
X_train = train.drop(labels= ['isAttended'], axis = 1)
X_test = test.drop(labels = ['isAttended'], axis = 1)


In [17]:
X_train.shape[1]

1928

In [18]:
X_train_columns = X_train.columns
X_test_columns = X_test.columns
y_train_columns = y_train.columns
y_test_columns = y_test.columns

In [17]:
#X_train = X_train.to_numpy(dtype= 'float32')
#X_test = X_test.to_numpy(dtype = 'float32')
#y_train = y_train.to_numpy(dtype = 'float32')
#y_test = y_test.to_numpy(dtype = 'float32')

In [19]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [ ]:
X_train = torch.tensor(X_train, dtype= torch.float32, requires_grad= True)
X_test =  torch.tensor(X_test, dtype = torch.float32, requires_grad= True)
y_train = torch.tensor(y_train, dtype = torch.float32, requires_grad= True)
y_test =  torch.tensor(y_test,dtype = torch.float32, requires_grad= True)

0
4
6
7
9
10
34
